In [98]:
import json
import pandas as pd

from glob import glob
from collections import defaultdict

In [99]:
traces = glob("./logs/bigscience/bloom-1b1/**/*.json", recursive=True)
total_durs = {}

for trace in traces:
    with open(trace) as f:
        json_dict = json.load(f)

    total_dur = {}

    for event in json_dict["traceEvents"]:
        if not "dur" in event:
            continue

        cat = event["cat"]
        dur = event["dur"]
        name = event["name"]

        if not dur:
            continue

        if not cat in total_dur:
            total_dur[cat] = defaultdict(int)

        # if "aten::" in name:
        #     name = "ManipulateTensor"
        # elif "autograd::" in name:
        #     name = "CalculateGradient"

        total_dur[cat][name] += dur

    dirname = trace.split("/")[-2]
    total_durs[dirname] = total_dur
    print(dirname)

    for cat, names in total_dur.items():
        cat_dur = 0
        print(f"  {cat}")
        for idx, (name, dur) in enumerate(names.items()):
            print(f"    [{idx+1}] {name} --> {dur}us")
            cat_dur += dur
        print(f"    [+] Total --> {cat_dur}us")

np8-bs32
  cpu_op
    [1] autograd::engine::evaluate_function: DivBackward0 --> 104us
    [2] DivBackward0 --> 82us
    [3] aten::div --> 186us
    [4] autograd::engine::evaluate_function: NllLossBackward0 --> 176us
    [5] NllLossBackward0 --> 145us
    [6] aten::nll_loss_backward --> 137us
    [7] aten::zero_ --> 186198us
    [8] aten::fill_ --> 184042us
    [9] autograd::engine::evaluate_function: LogSoftmaxBackward0 --> 93us
    [10] LogSoftmaxBackward0 --> 77us
    [11] aten::_log_softmax_backward_data --> 59us
    [12] autograd::engine::evaluate_function: ViewBackward0 --> 15498us
    [13] ViewBackward0 --> 13534us
    [14] aten::reshape --> 19364us
    [15] aten::_reshape_alias --> 1133us
    [16] autograd::engine::evaluate_function: CloneBackward0 --> 621us
    [17] CloneBackward0 --> 44us
    [18] autograd::engine::evaluate_function: SliceBackward0 --> 283346us
    [19] SliceBackward0 --> 282658us
    [20] aten::slice_backward --> 282470us
    [21] aten::zeros --> 181744us
   

In [100]:
df = pd.DataFrame(total_durs["np1-bs32"]["cpu_op"].items())
df.columns = ["name", "dur"]
df

,name,dur
0,autograd::engine::evaluate_function: DivBackward0,110
1,DivBackward0,89
2,aten::div,181
3,autograd::engine::evaluate_function: NllLossBa...,202
4,NllLossBackward0,169
...,...,...
127,aten::_local_scalar_dense,102428
128,aten::_foreach_sqrt,3672
129,aten::_foreach_div_,1098
130,aten::_foreach_add,3955


In [101]:
targets = {"aten::": "ManipulateTensor", "autograd::": "CalculateGradient", "Backward": "Backward"}
for target, alt_name in targets.items():
    len_ori = len(df)
    df_temp = df[df["name"].str.contains(target)]
    df.loc[len_ori] = ({"name": alt_name, "dur": df_temp["dur"].sum()})
    df.drop(df_temp.index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"{alt_name} (length: {len_ori} --> {len(df)})")
df

ManipulateTensor (length: 132 --> 49)
CalculateGradient (length: 49 --> 26)
Backward (length: 26 --> 5)


,name,dur
0,detach_,1
1,GeLUFunction,8087
2,ManipulateTensor,4956180
3,CalculateGradient,3714136
4,Backward,2608973
